In [51]:
import pandas as pd 

data = pd.read_csv("broker.csv").replace('-', '', regex=True)
data["Größe in m² (Kataster)"] = pd.to_numeric(data["Größe in m² (Kataster)"], downcast="float")

data.head()

,technischer Schlüssel,Objektnummer,Bezirk,Ortsteil,Art der Grünanlage,Name der Grünanlage,Namenszusatz der Grünanlage,Baujahr,letztes Sanierungsjahr,Größe in m² (Kataster),Widmung,Nummer des Planungsraumes,Name Planungsraum,anrechenbar Grünflächenversorgung
0,00008100:00199415,147,MarzahnHellersdorf,Marzahn,Grünanlage,Barnimplatz und Weg entlang Straßenbahn,Havemannstraße und Wittenberger Straße,1994,,9660.200195,gewidmet,10010102,Havemannstr.,1
1,00008100:00199478,2300175,MarzahnHellersdorf,Hellersdorf,Grünanlage,Grünfläche (DEST),Oschatzer Ring,,,1048.000000,gewidmet,10020416,Böhlener Str.,1
2,00008100:0019947a,2301127,MarzahnHellersdorf,Hellersdorf,Grünanlage,Kurt Julius Goldstein Park,Riesaer/ Heidenauer Straße,,,35327.000000,gewidmet,10020517,AdeleSandrockStr.,1
3,00008100:00199457,204,MarzahnHellersdorf,Marzahn,Grünanlage,Landsberger Allee 565,Dreiecksfläche Ringenwalder/Brodowiner R,1996,,5802.000000,gewidmet,10010205,MarzahnOst,1
4,00008100:0019942f,211,MarzahnHellersdorf,Marzahn,Grünanlage,Blumberger Damm/ Wuhletalstraße,Grünanlage mit Teich ggü.Kemberger Str.,1986,,4553.799805,gewidmet,10010204,Wuhletalstr.,1
5,00008100:00199263,2306641,MarzahnHellersdorf,Marzahn,Grünanlage,Stadtplatz am Glambecker Ring,Glambecker Ring/Ringenwalder Straße,2010,,6018.000000,gewidmet,10010205,MarzahnOst,1
6,00008100:0019965a,2300153,MarzahnHellersdorf,Hellersdorf,Grünanlage,Rasendreieck AlteHellersdorfer Str.,Alte Hellersdorfer Straße,,,1257.000000,gewidmet,10020413,Gut Hellersdorf,0
7,00008100:00199463,2301129,MarzahnHellersdorf,Hellersdorf,Grünanlage,"Parktriologie, Flächen unter Freileitung",zw.Hönower Weiherk.u.UBahn,,,62170.000000,gewidmet,10020517,AdeleSandrockStr.,1
8,00008100:00199468,2300173,MarzahnHellersdorf,Hellersdorf,Grünanlage,"Hellersdorfer Graben, 173",(zw.Jugendclub U 5 u.CarolaNeher Str.),,,22973.000000,gewidmet,10020519,Boulevard Kastanienallee,1
9,00008100:00224a87,R0223906,MarzahnHellersdorf,Marzahn,Grünanlage,"MaxHerrmannStr. 6,",Rückauflassung zum 01.04.2018,2009,,3242.000000,gewidmet,10010206,Ringkolonnaden,0


In [2]:
totalGreens = data['Größe in m² (Kataster)'].sum()
print("%s square metre" % totalGreens)
print("%s hectare" % (totalGreens/10000))

61376304.0 square metre
6137.6304 hectare


In [3]:
len(data['Nummer des Planungsraumes'].unique().tolist())

412

In [4]:
list(filter(None, data['Name Planungsraum'].unique().tolist()))

['Havemannstr.',
 'Böhlener Str.',
 'AdeleSandrockStr.',
 'MarzahnOst',
 'Wuhletalstr.',
 'Gut Hellersdorf',
 'Boulevard Kastanienallee',
 'Ringkolonnaden',
 'AltMarzahn',
 'Springpfuhl',
 'MahlsdorfSüd',
 'Marzahner Chaussee',
 'AltMahlsdorf',
 'KaulsdorfNord I',
 'Gewerbegebiet Bitterfelder Str.',
 'AltBiesdorf',
 'KaulsdorfNord II',
 'BiesdorfSüd',
 'Landsberger Tor',
 'Hellersdorfer Promenade',
 'Buckower Ring',
 'MarzahnWest',
 'Oberfeldstr.',
 'KaulsdorfNord',
 'Schleipfuhl',
 'AltKaulsdorf',
 'Marzahner Promenade',
 'KaulsdorfSüd',
 'MahlsdorfNord',
 'Helle Mitte',
 'Alte Hellersdorfer Straße',
 'Gelbes Viertel',
 'Grazer Platz',
 'Volkspark (RudolfWildePark)',
 'NeuTempelhof',
 'Rathaus Tempelhof',
 'Eisenacher Str.',
 'Germaniagarten',
 'Rathausstr.',
 'FritzWernerStr.',
 'AltLichtenrade/Töpchiner Weg',
 'Kirchstr.',
 'Horstwalder Str./Paplitzer Str.',
 'JohnLockeStr.',
 'Ceciliengärten',
 'Marienhöhe',
 'Rotes Viertel',
 'Marienfelder Allee Nordwest',
 'Marienfelde Süd',
 'Na

In [5]:
data.columns

Index(['technischer Schlüssel', 'Objektnummer', 'Bezirk', 'Ortsteil',
       'Art der Grünanlage', 'Name der Grünanlage',
       'Namenszusatz der Grünanlage', 'Baujahr', 'letztes Sanierungsjahr',
       'Größe in m² (Kataster)', 'Widmung', 'Nummer des Planungsraumes',
       'Name Planungsraum', 'anrechenbar Grünflächenversorgung'],
      dtype='object')

In [6]:
data.groupby('Name Planungsraum', as_index=False).agg({"Größe in m² (Kataster)": "sum"})

,Name Planungsraum,Größe in m² (Kataster)
0,,39602.601562
1,Ackerstr.,11475.000000
2,Adamstr.,184327.000000
3,AdeleSandrockStr.,181548.000000
4,Adlershof West,40350.898438
5,Alexanderplatzviertel,122864.500000
6,Allende I,20186.900391
7,Allende II,20384.900391
8,AltBiesdorf,278738.093750
9,AltGatow,561499.000000


In [52]:
data.groupby('Name Planungsraum', as_index=False).size().reset_index(name='Anzahl der Grünflächen').sort_values(['Anzahl der Grünflächen'], ascending=False).style.hide_index()


Name Planungsraum,Anzahl der Grünflächen
Sewanstraße,37
Frohnau,31
Hakenfelde Nord,30
Dorf Wartenberg,28
PankowZentrum,25
Stralauer Halbinsel,22
AltMarzahn,21
Herzbergstraße,21
Heiligensee,20
Buchholz,19


In [23]:
data.groupby('Bezirk').size().reset_index(name='Anzahl der Grünflächen').sort_values(['Anzahl der Grünflächen'], ascending=False).style.hide_index()

Bezirk,Anzahl der Grünflächen
Lichtenberg,299
Pankow,277
MarzahnHellersdorf,246
Mitte,245
Spandau,238
FriedrichshainKreuzberg,214
TreptowKöpenick,198
Reinickendorf,192
SteglitzZehlendorf,188
CharlottenburgWilmersdorf,157


In [48]:
empty_PLR = data["Name Planungsraum"] == ""
data[empty_PLR]

,technischer Schlüssel,Objektnummer,Bezirk,Ortsteil,Art der Grünanlage,Name der Grünanlage,Namenszusatz der Grünanlage,Baujahr,letztes Sanierungsjahr,Größe in m² (Kataster),Widmung,Nummer des Planungsraumes,Name Planungsraum,anrechenbar Grünflächenversorgung
115,00008100:00223253,002951,MarzahnHellersdorf,,Grünanlage,"""Wiesenpark""",Grundstück Privat (am Wiesenpark),,,NaN,,,,0
1449,00008100:0022f8c0,400110,Mitte,,Grünanlage,FernradwanderwegBerlinKopenhagen südl. Festplatz,Gemarkung Charlottenburg WSVGelände,,,1855.000000,,,,0
1812,00008100:001c64ba,45010,TreptowKöpenick,Johannisthal,Grünanlage,Späthstraße 80 / 81 Arboretum Ba NA,Sen Stadt / Humboldt Uni,,,37589.000000,,NN,,0
1897,00008100:00273c04,PO24970,Mitte,,Grünanlage,Großer Tiergarten i,Insel im Tiergartengewässer,,,158.600006,gewidmet,,,0
2097,00008100:0028901c,36194,Reinickendorf,Tegel,Grünanlage,"Waidmannsluster Damm, Sackgasse",,,,NaN,,,,0
